##### based on https://github.com/jason9075/opencv-mosaic-data-aug by jason9075

In [31]:
import os
import json
import random

import numpy as np
import cv2

In [38]:
# FIXME
dataset_0 = 'ICDAR19_MLT'
dataset_1 = 'ICDAR19_MLT' 
dataset_2 = 'ICDAR17_Korean'
dataset_3 = 'ICDAR19_Korean'

TOTAL = 2000                   # 생성할 이미지 개수
OUTPUT_SIZE = (1024, 1024)  # Height, Width
SCALE_RANGE = (0.3, 0.7)    # 모자이크 4사분면 나누는 축. 4사진 다 똑같은 크기 하고싶으면 (0.5, 0.5)
FILTER_TINY = 0             # 2500면 50px * 50px 보다 작은 bbox는 제거

In [39]:
IMG_DIR_0 = os.path.join('/opt/ml/input/data/', dataset_0, 'images')
IMG_DIR_1 = os.path.join('/opt/ml/input/data/', dataset_1, 'images')
IMG_DIR_2 = os.path.join('/opt/ml/input/data/', dataset_2, 'images')
IMG_DIR_3 = os.path.join('/opt/ml/input/data/', dataset_3, 'images')

LABEL_DIR_0 = os.path.join('/opt/ml/input/data/', dataset_0, 'ufo', 'train.json')
LABEL_DIR_1 = os.path.join('/opt/ml/input/data/', dataset_1, 'ufo', 'train.json')
LABEL_DIR_2 = os.path.join('/opt/ml/input/data/', dataset_2, 'ufo', 'train.json')
LABEL_DIR_3 = os.path.join('/opt/ml/input/data/', dataset_3, 'ufo', 'train.json')

os.makedirs(os.path.join('/opt/ml/input/data/', 'mosaic', 'images'), exist_ok=True)
os.makedirs(os.path.join('/opt/ml/input/data/', 'mosaic', 'ufo'), exist_ok=True)

In [40]:
with open(LABEL_DIR_0, 'r', encoding='utf-8') as f:
    ufo_0 = json.load(f)['images']
ufo_image_names_0 = [x for x in ufo_0.keys()]

with open(LABEL_DIR_1, 'r', encoding='utf-8') as f:
    ufo_1 = json.load(f)['images']
ufo_image_names_1 = [x for x in ufo_1.keys()]

with open(LABEL_DIR_2, 'r', encoding='utf-8') as f:
    ufo_2 = json.load(f)['images']
ufo_image_names_2 = [x for x in ufo_2.keys()]

with open(LABEL_DIR_3, 'r', encoding='utf-8') as f:
    ufo_3 = json.load(f)['images']
ufo_image_names_3 = [x for x in ufo_3.keys()]

ufos = [ufo_0, ufo_1, ufo_2, ufo_3]
ufo_image_names = [ufo_image_names_0, ufo_image_names_1, ufo_image_names_2, ufo_image_names_3]
img_dirs = [IMG_DIR_0, IMG_DIR_1, IMG_DIR_2, IMG_DIR_3]

In [41]:
output_img = np.zeros([OUTPUT_SIZE[0], OUTPUT_SIZE[1], 3], dtype=np.uint8)

license_tag = dict(usability=True, public=True,
                   commercial=True, type='CC-BY-SA',
                   holder=None)
orientation = 'Horizontal'


new_anno = dict()

for i in range(TOTAL):
    
    # scale_x, scale_y는 SCALE_RANGE 사이의 숫자
    scale_x = SCALE_RANGE[0] + random.random() * (SCALE_RANGE[1] - SCALE_RANGE[0])
    scale_y = SCALE_RANGE[0] + random.random() * (SCALE_RANGE[1] - SCALE_RANGE[0])
    divid_point_x = int(scale_x * OUTPUT_SIZE[1])
    divid_point_y = int(scale_y * OUTPUT_SIZE[0])

    temp_anno = dict(img_h=OUTPUT_SIZE[0], img_w=OUTPUT_SIZE[1],
                     tags=None, license_tag=license_tag)
    temp_words = dict()
    counter = 0
    
    for j in range(4):
        random_index = random.randrange(4)

        index = random.randrange(len(ufos[random_index]))
        while ((random_index == 0 or random_index == 1) and (index == 102)):
            index = random.randrange(len(ufos[random_index]))
            
        key = ufo_image_names[random_index][index]
        value = ufos[random_index][key]
        img = cv2.imread(os.path.join(img_dirs[random_index], key))
        
        try:
            img_height, img_width, _ = img.shape
        except:
            print("아래 파일을 살펴보세요~")
            print(os.path.join(img_dirs[random_index], key))
            
        
        # top-left
        if j == 0:
            try:
                img = cv2.resize(img, (divid_point_x, divid_point_y))
            except:
                print(f"image shape : {img.shape}")
                print(f"filename : {cv2.imread(os.path.join(img_dirs[random_index], key))}")
            output_img[:divid_point_y, :divid_point_x, :] = img

            for word in value['words'].items():
                word = word[1]
                a0, a1 = word['points'][0][0], word['points'][0][1]
                b0, b1 = word['points'][1][0], word['points'][1][1]
                c0, c1 = word['points'][2][0], word['points'][2][1]
                d0, d1 = word['points'][3][0], word['points'][3][1]
                
                a0 = float(int(a0*divid_point_x / img_width))
                a1 = float(int(a1*divid_point_y / img_height))
                
                b0 = float(int(b0*divid_point_x / img_width))
                b1 = float(int(b1*divid_point_y / img_height))
                
                c0 = float(int(c0*divid_point_x / img_width))
                c1 = float(int(c1*divid_point_y / img_height))
                
                d0 = float(int(d0*divid_point_x / img_width))
                d1 = float(int(d1*divid_point_y / img_height))

                if (c0-a0)*(c1-a1) < FILTER_TINY:
                    continue
                temp_words[counter] = dict(points=[[a0,a1],[b0,b1],[c0,c1],[d0,d1]],
                                        transcription=word['transcription'],
                                        language=word['language'],
                                        illegibility=word['illegibility'],
                                        orientation=word['orientation'],
                                        word_tags=word['word_tags'])
                counter += 1

        # top-right
        elif j == 1:
            try:
                img = cv2.resize(img, (OUTPUT_SIZE[1] - divid_point_x, divid_point_y))
            except:
                print(f"image shape : {img.shape}")
                print(f"filename : {cv2.imread(os.path.join(img_dirs[random_index], key))}")
                
            output_img[:divid_point_y, divid_point_x:OUTPUT_SIZE[1], :] = img

            for word in value['words'].items():
                word = word[1]
                a0, a1 = word['points'][0][0], word['points'][0][1]
                b0, b1 = word['points'][1][0], word['points'][1][1]
                c0, c1 = word['points'][2][0], word['points'][2][1]
                d0, d1 = word['points'][3][0], word['points'][3][1]
                
                a0 = float(int(divid_point_x + (a0*(OUTPUT_SIZE[1]-divid_point_x) / img_width)))
                a1 = float(int(a1*divid_point_y / img_height))
                
                b0 = float(int(divid_point_x + (b0*(OUTPUT_SIZE[1]-divid_point_x) / img_width)))
                b1 = float(int(b1*divid_point_y / img_height))
                
                c0 = float(int(divid_point_x + (c0*(OUTPUT_SIZE[1]-divid_point_x) / img_width)))
                c1 = float(int(c1*divid_point_y / img_height))
                
                d0 = float(int(divid_point_x + (d0*(OUTPUT_SIZE[1]-divid_point_x) / img_width)))
                d1 = float(int(d1*divid_point_y / img_height))
                
                if (c0-a0)*(c1-a1) < FILTER_TINY:
                    continue
                temp_words[counter] = dict(points=[[a0,a1],[b0,b1],[c0,c1],[d0,d1]],
                                        transcription=word['transcription'],
                                        language=word['language'],
                                        illegibility=word['illegibility'],
                                        orientation=word['orientation'],
                                        word_tags=word['word_tags'])
                counter += 1

        # bottom-left
        elif j == 2:
            try:
                img = cv2.resize(img, (divid_point_x, OUTPUT_SIZE[0] - divid_point_y))
            except:
                print(f"image shape : {img.shape}")
                print(f"filename : {cv2.imread(os.path.join(img_dirs[random_index], key))}")
                
            output_img[divid_point_y:OUTPUT_SIZE[0], :divid_point_x, :] = img

            for word in value['words'].items():
                word = word[1]
                a0, a1 = word['points'][0][0], word['points'][0][1]
                b0, b1 = word['points'][1][0], word['points'][1][1]
                c0, c1 = word['points'][2][0], word['points'][2][1]
                d0, d1 = word['points'][3][0], word['points'][3][1]
    
                a0 = float(int(a0*divid_point_x / img_width))
                a1 = float(int(divid_point_y + (a1*(OUTPUT_SIZE[0]-divid_point_y) / img_height)))
                
                b0 = float(int(b0*divid_point_x / img_width))
                b1 = float(int(divid_point_y + (b1*(OUTPUT_SIZE[0]-divid_point_y) / img_height)))
                
                c0 = float(int(c0*divid_point_x / img_width))
                c1 = float(int(divid_point_y + (c1*(OUTPUT_SIZE[0]-divid_point_y) / img_height)))
                
                d0 = float(int(d0*divid_point_x / img_width))
                d1 = float(int(divid_point_y + (d1*(OUTPUT_SIZE[0]-divid_point_y) / img_height)))

                if (c0-a0)*(c1-a1) < FILTER_TINY:
                    continue
                temp_words[counter] = dict(points=[[a0,a1],[b0,b1],[c0,c1],[d0,d1]],
                                        transcription=word['transcription'],
                                        language=word['language'],
                                        illegibility=word['illegibility'],
                                        orientation=word['orientation'],
                                        word_tags=word['word_tags'])
                counter += 1

        # bottom-right
        else:
            try:
                img = cv2.resize(img, (OUTPUT_SIZE[1] - divid_point_x, OUTPUT_SIZE[0] - divid_point_y))
            except:
                print(f"image shape : {img.shape}")
                print(f"filename : {cv2.imread(os.path.join(img_dirs[random_index], key))}")
            output_img[divid_point_y:OUTPUT_SIZE[0], divid_point_x:OUTPUT_SIZE[1], :] = img

            for word in value['words'].items():
                word = word[1]
                a0, a1 = word['points'][0][0], word['points'][0][1]
                b0, b1 = word['points'][1][0], word['points'][1][1]
                c0, c1 = word['points'][2][0], word['points'][2][1]
                d0, d1 = word['points'][3][0], word['points'][3][1]
                
                a0 = float(int(divid_point_x + (a0*(OUTPUT_SIZE[1]-divid_point_x) / img_width)))
                a1 = float(int(divid_point_y + (a1*(OUTPUT_SIZE[0]-divid_point_y) / img_height)))
                
                b0 = float(int(divid_point_x + (b0*(OUTPUT_SIZE[1]-divid_point_x) / img_width)))
                b1 = float(int(divid_point_y + (b1*(OUTPUT_SIZE[0]-divid_point_y) / img_height)))
                
                c0 = float(int(divid_point_x + (c0*(OUTPUT_SIZE[1]-divid_point_x) / img_width)))
                c1 = float(int(divid_point_y + (c1*(OUTPUT_SIZE[0]-divid_point_y) / img_height)))
                
                d0 = float(int(divid_point_x + (d0*(OUTPUT_SIZE[1]-divid_point_x) / img_width)))
                d1 = float(int(divid_point_y + (d1*(OUTPUT_SIZE[0]-divid_point_y) / img_height)))
            
                if (c0-a0)*(c1-a1) < FILTER_TINY:
                    continue
                temp_words[counter] = dict(points=[[a0,a1],[b0,b1],[c0,c1],[d0,d1]],
                                        transcription=word['transcription'],
                                        language=word['language'],
                                        illegibility=word['illegibility'],
                                        orientation=word['orientation'],
                                        word_tags=word['word_tags'])
                counter += 1
    
    # save image
    image_name = f'mosaic_{i}.jpg'
    cv2.imwrite(os.path.join('/opt/ml/input/data/', 'mosaic', 'images', image_name), output_img)
    
 
    temp_anno['words'] = temp_words
    new_anno[image_name] = temp_anno


# create ufo train.json
anno = dict(images=new_anno)

with open(os.path.join('/opt/ml/input/data/', 'mosaic', 'ufo', 'train.json'), 'w', encoding='utf-8') as f:
    json.dump(anno, f, indent=4)
    

In [42]:
# 모자이크 한 사진 위에 GT 바운딩박스 그려서 보여주기. opt/ml/ 에 생성
for word in temp_words.values():
    points = np.array(word['points'], dtype=np.int32) 
    cv2.polylines(output_img, [points], True, (0,255,0), 2)
cv2.imwrite('/opt/ml/mosaic.jpg', output_img)

True